# Fizz Buzz Benchmark Example
The explanation how to calc benchmark. (created by @sadahiroyoshi)

## Preparation
import libs & load config files.

In [ ]:
import platform
import pandas as pd
import numpy as np
import tool.language as language
import tool.benchmark as benchmark

In [ ]:
# load all languages in conf/languages.csv.
langs = language.load_languages()

## Print Your Device Info
check your environment before execute.

In [ ]:
# print OS info of your device.
print( "platform.system()={}".format( platform.system() ))
print( "platform.version()={}".format( platform.version() ))
print( "platform.release()={}".format( platform.release() ))

In [ ]:
# print all versions of each langs on your device.
for _lang in langs:
    language.print_version(_lang)

## Run Method

run the method which is wanted to calculate a benchmark.

In [ ]:
# calculate all benchmarks of 'println'
method = 'println'
n = 100_000
r = 10

results_by_langs = {}
_debug = True
for _lang in langs:
    # if `debug=True`, you can check actual cmd line in `benchmark.run`
    results_by_langs[_lang] = benchmark.run(method, _lang, n, r, debug=_debug)
    _debug = False # `debug=True` only at first exec

`benchmark.run` is internally running method of each langs via CLI. The overhead time when staring CLI, may much effects execution time of this method.

So after `benchmark.run`, also run the method 'nothing'. this method is 'do nothing' or empty script file of each langs.

You will get real execution time by substracting the 'nothing' time. 

In [ ]:
# also run scripts 'nothing' (to measure execution time of lang cmd itself)
results_by_langs_nothing = {}
for _lang in langs:
    results_by_langs_nothing[_lang] = benchmark.run('nothing', _lang, n, r)

In [ ]:
# calculate avg of the method & 'nothing'.
_avgs = [
    sum(results_by_langs[_lang]) / r for _lang in langs
]
_avgs_nothing = [
    sum(results_by_langs_nothing[_lang]) / r for _lang in langs
]

df = pd.DataFrame({
    method: _avgs,
    'nothing': _avgs_nothing,
}, index=langs)
df.index.name = 'language'

In [ ]:
# substracting the 'nothing' time.
df['real'] = df[method] - df['nothing']

## Data Plot
finally, plot these results.

In [ ]:
df[method].plot.barh()

In [ ]:
df['nothing'].plot.barh()

In [ ]:
df['real'].plot.barh()